In [1]:
import pandas as pd
from pandasql import sqldf
import datetime 

In [2]:
Get_Alarms_History=pd.read_json('https://www.oref.org.il//Shared/Ajax/GetAlarmsHistory.aspx?lang=he&fromDate=7.10.2023&toDate=&mode=0')
GetAlertCategories=pd.read_json('https://www.oref.org.il//Shared/Ajax/GetAlertCategories.aspx')
GetCities=pd.read_json('https://www.oref.org.il/Shared/Ajax/GetCities.aspx?lang=he')
GetDistricts=pd.read_json('https://www.oref.org.il/Shared/Ajax/GetDistricts.aspx?lang=he')
AlertsHistory=pd.read_json('https://www.oref.org.il/WarningMessages/History/AlertsHistory.json')


In [3]:
Get_Alarms_History.columns

Index(['data', 'date', 'time', 'alertDate', 'category', 'category_desc',
       'matrix_id', 'rid'],
      dtype='object')

In [4]:
GetDistricts.columns

Index(['label', 'value', 'id', 'areaid', 'areaname', 'label_he', 'migun_time'], dtype='object')

In [5]:
GetCities.columns

Index(['label', 'rashut', 'value', 'id', 'areaid', 'mixname', 'color'], dtype='object')

In [6]:
days=[i for i in range (7,datetime.date.today().day+1)]
current_month=datetime.date.today().month

from datetime import datetime

full_dates=[datetime(2023, current_month, i).strftime("%d.%m.%Y") for i in days]
full_dates

['07.10.2023',
 '08.10.2023',
 '09.10.2023',
 '10.10.2023',
 '11.10.2023',
 '12.10.2023',
 '13.10.2023',
 '14.10.2023',
 '15.10.2023',
 '16.10.2023',
 '17.10.2023',
 '18.10.2023',
 '19.10.2023',
 '20.10.2023',
 '21.10.2023',
 '22.10.2023',
 '23.10.2023']

In [7]:
df=pd.read_json('https://www.oref.org.il//Shared/Ajax/GetAlarmsHistory.aspx?lang=he&fromDate={}&toDate={}&mode=0'.format(full_dates[0],full_dates[0]))
empty_df = pd.DataFrame(columns=df.columns)

for i in full_dates:
    tmp=pd.read_json('https://www.oref.org.il//Shared/Ajax/GetAlarmsHistory.aspx?lang=he&fromDate={}&toDate={}&mode=0'.format(i,i))
    empty_df=empty_df.append(tmp, ignore_index=True)

empty_df['alertDate']= pd.to_datetime(empty_df['alertDate']).dt.date
empty_df

,data,date,time,alertDate,category,category_desc,matrix_id,rid
0,"שדרות, איבים, ניר עם",2023-07-10,23:42:45,2023-10-07,1,ירי רקטות וטילים,1,20550
1,שוקדה,2023-07-10,23:13:53,2023-10-07,1,ירי רקטות וטילים,1,20549
2,בארי,2023-07-10,23:13:53,2023-10-07,1,ירי רקטות וטילים,1,20548
3,תקומה,2023-07-10,22:43:18,2023-10-07,1,ירי רקטות וטילים,1,20547
4,תקומה וחוות יזרעם,2023-07-10,22:43:18,2023-10-07,1,ירי רקטות וטילים,1,20546
...,...,...,...,...,...,...,...,...
4677,יד מרדכי,2023-10-23,11:30:21,2023-10-23,1,ירי רקטות וטילים,1,23214
4678,מבקיעים,2023-10-23,11:30:17,2023-10-23,1,ירי רקטות וטילים,1,23213
4679,בת הדר,2023-10-23,11:30:17,2023-10-23,1,ירי רקטות וטילים,1,23212
4680,אזור תעשייה הדרומי אשקלון,2023-10-23,11:30:06,2023-10-23,1,ירי רקטות וטילים,1,23211


In [8]:
final_table=sqldf('''

with base as(
select distinct
g1.rid,
g1.data as city,
g1.alertDate as date,
g1.time,
g1.category_desc,
g2.areaname,
g2.migun_time,
Case when g3.color in ("None","O") then 'כתום' 
when g3.color="Y" then 'צהוב' else 'ירוק' end as color

from empty_df g1 left join GetDistricts g2 on g1.data=g2.label_he
left join GetCities g3 on g2.id=g3.value and g2.areaid=g3.areaid)

select *
from base
order by date asc,time asc
''')

final_table

,rid,city,date,time,category_desc,areaname,migun_time,color
0,18554,שורש,2023-10-07,08:57:01,ירי רקטות וטילים,שפלת יהודה,90,צהוב
1,18541,כפר ורבורג,2023-10-07,08:57:04,ירי רקטות וטילים,לכיש,45,צהוב
2,18540,באר טוביה,2023-10-07,08:57:04,ירי רקטות וטילים,לכיש,45,צהוב
3,18539,אביגדור,2023-10-07,08:57:04,ירי רקטות וטילים,לכיש,45,צהוב
4,18562,קריית יערים,2023-10-07,08:57:29,ירי רקטות וטילים,שפלת יהודה,90,צהוב
...,...,...,...,...,...,...,...,...
4677,23225,באר שבע - מערב,2023-10-23,13:00:42,ירי רקטות וטילים,מרכז הנגב,60,ירוק
4678,23229,לקיה,2023-10-23,13:00:47,ירי רקטות וטילים,מרכז הנגב,60,צהוב
4679,23232,עומר,2023-10-23,13:01:00,ירי רקטות וטילים,מרכז הנגב,60,צהוב
4680,23231,באר שבע - מזרח,2023-10-23,13:01:00,ירי רקטות וטילים,מרכז הנגב,60,ירוק


In [9]:
try:
    final_table.to_csv('/Users/mayrozental/Desktop/pikud_oref_alarm.csv',index=False)
    print('Sucsses')
except:
    print('Error')

Sucsses
